**FCFS**
 stands for First-Come, First-Served and is a scheduling algorithm used in operating systems and computer science. It is a non-preemptive scheduling algorithm, which means that once a process starts executing, it will continue to do so until it finishes or is blocked.

In FCFS, the processes are executed in the order in which they arrive in the ready queue. The first process that arrives is the first to be executed, and so on. This can lead to a phenomenon known as the "convoy effect," where smaller processes are held up behind a larger process, causing potential delays.

While FCFS is simple and easy to implement, it may not be the most efficient scheduling algorithm, especially for systems with a mix of short and long processes. It can lead to longer average waiting times and turnaround times compared to other scheduling algorithms like Round Robin or Shortest Job First.

Overall, FCFS is a basic and straightforward scheduling algorithm that is easy to implement but may not always provide the best performance in terms of overall system efficiency.



In [ ]:
class Process:
    def __init__(self, process_id, arrival_time, priority, burst_time):
        self.process_id = process_id
        self.arrival_time = arrival_time
        self.priority = priority
        self.burst_time = burst_time
        self.start_time = 0
        self.finish_time = 0

def calculate_statistics(processes):
    throughput = len(processes)
    total_burst_time = sum(process.burst_time for process in processes)
    completion_times = [sum(process.burst_time for process in processes[:i+1]) for i in range(len(processes))]
    cpu_utilization = total_burst_time / completion_times[-1]
    waiting_times = [max(0, completion_times[i] - processes[i].arrival_time - processes[i].burst_time) for i in range(len(processes))]
    turnaround_times = [completion_times[i] - processes[i].arrival_time for i in range(len(processes))]
    response_times = [processes[i].start_time - processes[i].arrival_time for i in range(len(processes))]

    avg_waiting_time = sum(waiting_times) / len(processes)
    avg_turnaround_time = sum(turnaround_times) / len(processes)
    avg_response_time = sum(response_times) / len(processes)

    return throughput, cpu_utilization, avg_waiting_time, avg_turnaround_time, avg_response_time

def main():
    with open('list.txt', 'r') as file:
        lines = file.readlines()

    processes = []
    for line in lines:
        process_info = line.strip().split(', ')
        process = Process(process_info[0], int(process_info[1]), int(process_info[2]), int(process_info[3]))
        processes.append(process)

    current_time = 0
    for process in processes:
        process.start_time = max(current_time, process.arrival_time)
        process.finish_time = process.start_time + process.burst_time
        current_time = process.finish_time

    throughput, cpu_utilization, avg_waiting_time, avg_turnaround_time, avg_response_time = calculate_statistics(processes)

    with open('FCFS.txt', 'w') as output_file:
        output_file.write(f"Throughput: {throughput}\n")
        output_file.write(f"CPU Utilization: {cpu_utilization}\n")
        output_file.write(f"Average Waiting Time: {avg_waiting_time}\n")
        output_file.write(f"Average Turnaround Time: {avg_turnaround_time}\n")
        output_file.write(f"Average Response Time: {avg_response_time}\n")

        for process in processes:
            output_file.write(f"{process.process_id} {process.start_time} {process.finish_time}\n")

if __name__ == "__main__":
    main()


Preemptive Shortest Job First (SJF) is a scheduling algorithm that selects the shortest job from the ready queue for execution. In preemptive SJF, if a new job arrives with a shorter burst time than the remaining time of the currently executing job, the currently executing job is preempted and the new job is executed. This allows for better utilization of CPU and can result in lower average waiting times for processes.

In [ ]:
class Process:
    def __init__(self, pid, arrival_time, priority, burst_time):
        self.pid = pid
        self.arrival_time = arrival_time
        self.priority = priority
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.start_times = []
        self.finish_times = []
        self.waiting_time = 0
        self.turnaround_time = 0
        self.response_time = -1

    def execute(self, current_time):
        if self.response_time == -1:
            self.response_time = current_time
        self.start_times.append(current_time)
        if self.remaining_time > 0:
            self.remaining_time -= 1
        current_time += 1
        self.finish_times.append(current_time)
        return current_time

def preemptive_sjf_scheduling(processes):
    current_time = 0
    completed_processes = []
    while True:
        if len(processes) == 0 and all(process.remaining_time == 0 for process in completed_processes):
            break
        available_processes = [process for process in processes if process.arrival_time <= current_time and process.remaining_time > 0]
        if len(available_processes) == 0:
            current_time += 1
        else:
            available_processes.sort(key=lambda x: x.remaining_time)
            executing_process = available_processes[0]
            current_time = executing_process.execute(current_time)
            if executing_process.remaining_time == 0:
                completed_processes.append(executing_process)
                processes.remove(executing_process)
    return completed_processes

def calculate_statistics(processes):
    waiting_times = [process.response_time - process.arrival_time for process in processes]
    turnaround_times = [process.finish_times[-1] - process.arrival_time for process in processes]
    response_times = waiting_times
    throughput = len(processes) / processes[-1].finish_times[-1]
    cpu_utilization = sum([process.burst_time for process in processes if process.burst_time > 0]) / processes[-1].finish_times[-1]
    avg_waiting_time = sum(waiting_times) / len(waiting_times)
    avg_turnaround_time = sum(turnaround_times) / len(turnaround_times)
    avg_response_time = sum(response_times) / len(response_times)
    return (throughput, cpu_utilization, avg_waiting_time, avg_turnaround_time, avg_response_time)

def main():
    processes = []
    with open('list.txt', 'r') as file:
        for line in file:
            data = line.strip().split(', ')
            processes.append(Process(data[0], int(data[1]), int(data[2]), int(data[3])))

    completed_processes = preemptive_sjf_scheduling(processes)
    throughput, cpu_utilization, avg_waiting_time, avg_turnaround_time, avg_response_time = calculate_statistics(completed_processes)

    with open('Preemptive_SJF.txt', 'w') as output_file:
        output_file.write(str(throughput) + "\n")
        output_file.write("{:.0%}".format(cpu_utilization) + "\n")
        output_file.write("{:.1f}".format(avg_waiting_time) + "\n")
        output_file.write("{:.1f}".format(avg_turnaround_time) + "\n")
        output_file.write("{:.1f}".format(avg_response_time) + "\n")
        for process in completed_processes:
            output_file.write(f"{process.pid} {process.start_times[0]} {process.finish_times[-1]}\n")

if __name__ == "__main__":
    main()

Non-preemptive Shortest Job First (SJF) is a scheduling algorithm that selects the shortest job from the ready queue for execution, but once a job starts executing, it is not preempted by a shorter job arriving. This means that the currently running job will continue until it finishes, even if a shorter job enters the ready queue. Non-preemptive SJF can lead to longer average waiting times for processes compared to preemptive SJF, as shorter jobs arriving later may have to wait until the currently running job finishes.

In [ ]:
class Process:
    def __init__(self, pid, arrival_time, priority, burst_time):
        self.pid = pid
        self.arrival_time = arrival_time
        self.priority = priority
        self.burst_time = burst_time
        self.start_time = 0
        self.finish_time = 0
        self.waiting_time = 0
        self.turnaround_time = 0
        self.response_time = -1

def non_preemptive_sjf_scheduling(processes):
    current_time = 0
    completed_processes = []
    total_waiting_time = 0
    total_turnaround_time = 0
    total_response_time = 0

    while len(processes) > 0:
        available_processes = [process for process in processes if process.arrival_time <= current_time]
        if len(available_processes) == 0:
            current_time += 1
        else:
            available_processes.sort(key=lambda x: x.burst_time)
            executing_process = available_processes[0]
            executing_process.start_time = current_time
            executing_process.finish_time = current_time + executing_process.burst_time
            executing_process.waiting_time = executing_process.start_time - executing_process.arrival_time
            executing_process.turnaround_time = executing_process.finish_time - executing_process.arrival_time
            if executing_process.response_time == -1:
                executing_process.response_time = executing_process.start_time - executing_process.arrival_time
            total_waiting_time += executing_process.waiting_time
            total_turnaround_time += executing_process.turnaround_time
            total_response_time += executing_process.response_time
            current_time = executing_process.finish_time
            completed_processes.append(executing_process)
            processes.remove(executing_process)

    throughput = len(completed_processes) / current_time
    cpu_utilization = sum(p.burst_time for p in completed_processes) / current_time
    avg_waiting_time = total_waiting_time / len(completed_processes)
    avg_turnaround_time = total_turnaround_time / len(completed_processes)
    avg_response_time = total_response_time / len(completed_processes)

    with open('Non_Preemptive_SJF.txt', 'w') as output_file:
        output_file.write(str(throughput) + "\n")
        output_file.write("{:.0%}".format(cpu_utilization) + "\n")
        output_file.write("{:.1f}".format(avg_waiting_time) + "\n")
        output_file.write("{:.1f}".format(avg_turnaround_time) + "\n")
        output_file.write("{:.1f}".format(avg_response_time) + "\n")

        for process in completed_processes:
            output_file.write(process.pid + ', ' + str(process.start_time) + ', ' + str(process.finish_time) + '\n')

def main():
    processes = []
    with open('list.txt', 'r') as file:
        for line in file:
            data = line.strip().split(', ')
            processes.append(Process(data[0], int(data[1]), int(data[2]), int(data[3])))

    non_preemptive_sjf_scheduling(processes)

if __name__ == "__main__":
    main()

Round Robin is a CPU scheduling algorithm that is designed for time-sharing systems. In this algorithm, each process is assigned a fixed time period, often referred to as a time quantum or time slice. When a process is scheduled to run, it is allowed to execute for the duration of its time quantum. If the process is not completed within the time quantum, it is preempted and placed at the back of the ready queue. The scheduler then selects the next process in the queue to run, and the cycle continues.

Round Robin is a simple and fair scheduling algorithm that ensures that all processes get an equal share of the CPU time. However, it may not be optimal for long-running processes or for systems with a mix of short and long processes, as it can lead to high context switch overhead.

In [ ]:
class Process:
    def __init__(self, pid, arrival, priority, burst):
        self.pid = pid
        self.arrival = int(arrival)
        self.priority = int(priority)
        self.burst = int(burst)
        self.remaining_burst = int(burst)
        self.start_time = -1
        self.finish_time = 0
        self.waiting_time = 0
        self.response_time = -1  # Initialize response time

    def update_waiting_and_response_time(self, current_time):
        if self.remaining_burst > 0:
            self.waiting_time = current_time - self.arrival - (self.burst - self.remaining_burst)
        if self.response_time == -1:
            self.response_time = current_time - self.arrival

def read_input(file_name):
    processes = []
    with open(file_name, 'r') as file:
        for line in file:
            pid, arrival, priority, burst = line.strip().split(', ')
            processes.append(Process(pid, arrival, priority, burst))
    return processes

def round_robin(processes, time_quantum):
    time = 0
    queue = []
    finished_processes = []
    processes.sort(key=lambda x: x.arrival)

    while processes or queue:
        while processes and processes[0].arrival <= time:
            queue.append(processes.pop(0))

        if queue:
            current_process = queue.pop(0)
            if current_process.start_time == -1:
                current_process.start_time = time
            execution_time = min(current_process.remaining_burst, time_quantum)
            current_process.remaining_burst -= execution_time
            time += execution_time

            if current_process.remaining_burst > 0:
                current_process.update_waiting_and_response_time(time)
                queue.append(current_process)
            else:
                current_process.finish_time = time
                current_process.update_waiting_and_response_time(time)
                finished_processes.append(current_process)
        else:
            time += 1

    return finished_processes

def calculate_statistics(processes):
    total_waiting_time = sum(p.waiting_time for p in processes)
    total_turnaround_time = sum(p.finish_time - p.arrival for p in processes)
    total_burst_time = sum(p.burst for p in processes)
    total_response_time = sum(p.response_time for p in processes)

    return {
        'Throughput': len(processes) / max(p.finish_time for p in processes),
        'CPU Utilization': (total_burst_time / max(p.finish_time for p in processes)) * 100,
        'Average Waiting Time': total_waiting_time / len(processes),
        'Average Turnaround Time': total_turnaround_time / len(processes),
        'Average Response Time': total_response_time / len(processes)
    }

def write_output(file_name, processes, statistics):
    with open(file_name, 'w') as file:
        file.write(f"{statistics['Throughput']:.3f}\n")
        file.write(f"{statistics['CPU Utilization']:.0f}%\n")
        file.write(f"{statistics['Average Waiting Time']:.1f}\n")
        file.write(f"{statistics['Average Turnaround Time']:.1f}\n")
        file.write(f"{statistics['Average Response Time']:.1f}\n")

        for process in processes:
            file.write(f"{process.pid}, {process.start_time}, {process.finish_time}\n")

def main():
    time_quantum =int(input("give me the quantum: "))  # Define the time quantum here
    processes = read_input('list.txt')
    finished_processes = round_robin(processes, time_quantum)
    statistics = calculate_statistics(finished_processes)
    write_output('Round_Robin.txt', finished_processes, statistics)

if __name__ == "__main__":
    main()

give me the quantum: 4


Preemptive Priority is a CPU scheduling algorithm where each process is assigned a priority, and the CPU is allocated to the process with the highest priority. In a preemptive priority scheduling, if a new process arrives with a higher priority than the currently running process, the currently running process is preempted, and the new process with the higher priority is allowed to run.

This algorithm ensures that high priority processes are given preference and can be executed as soon as they arrive. However, it can lead to starvation of lower priority processes if higher priority processes continue to arrive frequently. Additionally, care must be taken to avoid priority inversion, where a low-priority process holds a resource that a high-priority process needs, causing the high-priority process to be blocked.

In [ ]:
class Process:
    def __init__(self, process_id, arrival_time, priority, cpu_burst):
        self.process_id = process_id
        self.arrival_time = arrival_time
        self.priority = priority
        self.cpu_burst = cpu_burst
        self.start_execution_time = 0  # Added attribute for start execution time
        self.finish_execution_time = 0  # Added attribute for finish execution time

# Function to calculate Priority with Preemption statistics and generate output file
def calculate_priority_with_preemption_stats(processes):
    processes.sort(key=lambda x: (x.arrival_time, x.priority))  # Sort processes based on arrival time and priority
    waiting_times = [0] * len(processes)
    turnaround_times = [0] * len(processes)
    response_times = [0] * len(processes)
    finish_times = [0] * len(processes)

    current_time = 0
    remaining_burst = [process.cpu_burst for process in processes]
    executed_processes = set()

    while any(burst > 0 for burst in remaining_burst):
        available_processes = [i for i, burst in enumerate(remaining_burst) if burst > 0 and processes[i].arrival_time <= current_time]
        if available_processes:
            highest_priority_process = min(available_processes, key=lambda x: processes[x].priority)
            if highest_priority_process not in executed_processes:
                response_times[highest_priority_process] = current_time - processes[highest_priority_process].arrival_time
                executed_processes.add(highest_priority_process)

            if remaining_burst[highest_priority_process] == processes[highest_priority_process].cpu_burst:
                processes[highest_priority_process].start_execution_time = current_time  # Set start execution time for the process

            remaining_burst[highest_priority_process] -= 1
            if remaining_burst[highest_priority_process] == 0:
                finish_times[highest_priority_process] = current_time + 1
                processes[highest_priority_process].finish_execution_time = finish_times[highest_priority_process]  # Set finish execution time for the process
                turnaround_times[highest_priority_process] = finish_times[highest_priority_process] - processes[highest_priority_process].arrival_time
            current_time += 1
        else:
            current_time += 1

    throughput = len(processes) / max(finish_times)
    total_cpu_time = sum(process.cpu_burst for process in processes)
    cpu_utilization = total_cpu_time / max(finish_times)
    avg_waiting_time = sum(turnaround - burst for turnaround, burst in zip(turnaround_times, [process.cpu_burst for process in processes])) / len(processes)
    avg_turnaround_time = sum(turnaround_times) / len(processes)
    avg_response_time = sum(response_times) / len(processes)

    output_file = open('Priority_with_preemption.txt', 'w')
    output_file.write(str(throughput) + '\n')
    output_file.write(str(cpu_utilization*100) + '%\n')
    output_file.write(str(avg_waiting_time) + '\n')
    output_file.write(str(avg_turnaround_time) + '\n')
    output_file.write(str(avg_response_time) + '\n')

    for process in processes:
        output_file.write(f"{process.process_id}, {process.start_execution_time}, {process.finish_execution_time}\n")

    output_file.close()

# Read input from list.txt file
def read_input_file(file_path):
    processes = []
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split(', ')
            process = Process(data[0], int(data[1]), int(data[2]), int(data[3]))
            processes.append(process)
    return processes

input_file_path = 'list.txt'
processes = read_input_file(input_file_path)
calculate_priority_with_preemption_stats(processes)

Non-preemptive priority scheduling is a CPU scheduling algorithm in which each process is assigned a priority, and the CPU is allocated to the process with the highest priority. Once a process starts executing, it continues until it completes or blocks for I/O, regardless of whether a higher priority process arrives.

In non-preemptive priority scheduling, the CPU is not taken away from a running process to give it to a higher priority process that arrives later. This means that if a lower priority process is currently running, it will continue to run until it finishes, even if a higher priority process becomes available.

One potential issue with non-preemptive priority scheduling is that it can lead to situations where lower priority processes may never get a chance to execute if higher priority processes continue to arrive. This is known as starvation.

Overall, non-preemptive priority scheduling is simple to implement but may not be as responsive as preemptive priority scheduling, especially in scenarios where higher priority processes need to be executed promptly.

In [ ]:
class Process:
    def __init__(self, process_id, arrival_time, priority, burst_time):
        self.process_id = process_id
        self.arrival_time = arrival_time
        self.priority = priority
        self.burst_time = burst_time
        self.start_time = 0
        self.finish_time = 0

def calculate_statistics(processes):
    current_time = 0
    total_burst_time = sum(process.burst_time for process in processes)
    completion_times = [0] * len(processes)
    remaining_burst_times = [process.burst_time for process in processes]
    queue = []

    processes.sort(key=lambda x: (x.arrival_time, x.priority))
    for process in processes:
        process.start_time = current_time
        current_time += process.burst_time
        process.finish_time = current_time
        completion_times[processes.index(process)] = current_time

    cpu_utilization = total_burst_time / current_time

    waiting_times = [max(0, completion_times[i] - processes[i].arrival_time - processes[i].burst_time) for i in range(len(processes))]
    turnaround_times = [completion_times[i] - processes[i].arrival_time for i in range(len(processes))]
    response_times = [process.start_time - process.arrival_time for process in processes]

    avg_waiting_time = sum(waiting_times) / len(processes)
    avg_turnaround_time = sum(turnaround_times) / len(processes)
    avg_response_time = sum(response_times) / len(processes)

    return current_time, cpu_utilization, avg_waiting_time, avg_turnaround_time, avg_response_time

def main():
    with open('list.txt', 'r') as file:
        lines = file.readlines()

    processes = []
    for line in lines:
        process_info = line.strip().split(', ')
        process = Process(process_info[0], int(process_info[1]), int(process_info[2]), int(process_info[3]))
        processes.append(process)

    total_time, cpu_utilization, avg_waiting_time, avg_turnaround_time, avg_response_time = calculate_statistics(processes)

    with open('NonPreemptivePriority.txt', 'w') as output_file:
        output_file.write(f"Throughput: {len(processes)}\n")
        output_file.write(f"CPU Utilization: {cpu_utilization}\n")
        output_file.write(f"Average Waiting Time: {avg_waiting_time}\n")
        output_file.write(f"Average Turnaround Time: {avg_turnaround_time}\n")
        output_file.write(f"Average Response Time: {avg_response_time}\n")

        for process in processes:
            output_file.write(f"{process.process_id} {process.start_time} {process.finish_time}\n")

if __name__ == "__main__":
    main()
